In [205]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# add quotes before and after <style>

### Load packages

In [1]:
import nltk
import datetime
import re
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import lxml
from io import StringIO
from collections import Counter
from sklearn.decomposition import PCA
import pickle
import json
import os

import seaborn as sns
sns.set_style("darkgrid")
%pylab inline

#visualization
from IPython.display import Image
import pydotplus
from sklearn.tree import export_graphviz

# visualization
%matplotlib inline
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import scipy.stats as stats
from sklearn.metrics import roc_curve, auc
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

In [3]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB

from textblob import TextBlob


### Load Datasets

In [4]:
# Load in Review Data Separately

In [5]:
review1 = np.load(("data/review1.pkl"))

In [6]:
review2 = np.load(("data/review2.pkl"))

In [7]:
review3 = np.load(("data/review3.pkl"))

In [8]:
review4 = np.load(("data/review4.pkl"))

In [9]:
review5 = np.load(("data/review5.pkl"))

In [10]:
review1.shape

(1000000, 9)

In [11]:
# Concatenate files
review = pd.concat([review1, review2, review3, review4, review5], ignore_index=True)

In [12]:
review.shape

(4736897, 9)

In [13]:
#sorted(review.date, reverse=True)

In [2]:
# All other files

business = np.load("data/business.pkl")
# tips = np.load("data/tips.pkl")
# checkin = np.load("data/checkin.pkl")
user_small = np.load('data/user_small.pkl')

In [14]:
business.iloc[25]['attributes']

{'Alcohol': 'full_bar',
 'Ambience': {'casual': False,
  'classy': False,
  'hipster': False,
  'intimate': False,
  'romantic': False,
  'touristy': False,
  'trendy': False,
  'upscale': False},
 'BestNights': {'friday': False,
  'monday': False,
  'saturday': True,
  'sunday': False,
  'thursday': False,
  'tuesday': False,
  'wednesday': False},
 'BikeParking': True,
 'BusinessAcceptsCreditCards': True,
 'BusinessParking': {'garage': False,
  'lot': False,
  'street': False,
  'valet': False},
 'CoatCheck': False,
 'DogsAllowed': False,
 'GoodForDancing': False,
 'HappyHour': False,
 'HasTV': True,
 'Music': {'background_music': False,
  'dj': False,
  'jukebox': True,
  'karaoke': False,
  'live': False,
  'no_music': False,
  'video': False},
 'NoiseLevel': 'loud',
 'OutdoorSeating': False,
 'RestaurantsGoodForGroups': True,
 'RestaurantsPriceRange2': 2,
 'Smoking': 'outdoor',
 'WheelchairAccessible': False}

In [14]:
# earliest elite status

user_small['first_yr_elite'] = [min(x) if len(x) > 0 else 99999 for x in user_small.elite.tolist()]

In [236]:
user_small.head()

,user_id,elite,yelping_since,first_yr_elite
0,lsSiIjAKVl-QRxKjRErBeg,"[2014, 2016, 2013, 2011, 2012, 2015, 2010, 2017]",2010-07-13,2010
1,om5ZiponkpRqUNa3pVPiRg,"[2014, 2017, 2011, 2012, 2015, 2009, 2013, 200...",2006-01-18,2006
2,-lGwMGHMC_XihFJNKCJNRg,[],2014-10-31,99999
3,D-ydMTPGWXTVm4_jjp0k9g,"[2016, 2017]",2013-04-01,2016
4,PcvbBOCOcs6_suRDH7TSTg,"[2017, 2016]",2012-08-16,2016


In [15]:
 user_small.drop('elite', axis=1, inplace = True)

In [16]:
tips.head()

,business_id,date,likes,text,user_id
0,tJRDll5yqpZwehenzE2cSg,2012-07-15,0,Get here early enough to have dinner.,zcTZk7OG8ovAmh_fenH21g
1,jH19V2I9fIslnNhDzPmdkA,2015-08-12,0,Great breakfast large portions and friendly wa...,ZcLKXikTHYOnYt5VYRO5sg
2,dAa0hB2yrnHzVmsCkN4YvQ,2014-06-20,0,Nice place. Great staff. A fixture in the tow...,oaYhjqBbh18ZhU0bpyzSuw
3,dAa0hB2yrnHzVmsCkN4YvQ,2016-10-12,0,Happy hour 5-7 Monday - Friday,ulQ8Nyj7jCUR8M83SUMoRQ
4,ESzO3Av0b1_TzKOiqzbQYQ,2017-01-28,0,"Parking is a premium, keep circling, you will ...",ulQ8Nyj7jCUR8M83SUMoRQ


In [17]:
business.head(3)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,691 Richmond Rd,"{'RestaurantsPriceRange2': 2, 'BusinessParking...",YDf95gJZaq05wvo7hTQbbQ,"[Shopping, Shopping Centers]",Richmond Heights,"{'Monday': '10:00-21:00', 'Tuesday': '10:00-21...",1,41.541716,-81.493116,Richmond Town Square,,44143,17,2.0,OH
1,2824 Milton Rd,"{'GoodForMeal': {'dessert': False, 'latenight'...",mLwM-h2YhXl2NCgdS84_Bw,"[Food, Soul Food, Convenience Stores, Restaura...",Charlotte,"{'Monday': '10:00-22:00', 'Tuesday': '10:00-22...",0,35.236870,-80.741976,South Florida Style Chicken & Ribs,Eastland,28215,4,4.5,NC
2,337 Danforth Avenue,"{'BusinessParking': {'garage': False, 'street'...",v2WhjAB3PIBA8J8VxG3wEg,"[Food, Coffee & Tea]",Toronto,"{'Monday': '10:00-19:00', 'Tuesday': '10:00-19...",0,43.677126,-79.353285,The Tea Emporium,Riverdale,M4K 1N7,7,4.5,ON


In [18]:
review.columns = ['business_id', 'cool', 'date', 'funny', 'review_id', 'user_stars', 'review_txt', 'useful', 'user_id']
review.head()

,business_id,cool,date,funny,review_id,user_stars,review_txt,useful,user_id
0,uYHaNptLzDLoV_JZ_MuzUA,0,2016-07-12,0,VfBHSwC5Vz_pbFluy07i9Q,5,My girlfriend and I stayed here for 3 nights a...,0,cjpdDjZyprfyDG3RlkVG3w
1,uYHaNptLzDLoV_JZ_MuzUA,0,2016-10-02,0,3zRpneRKDsOPq92tq7ybAA,3,If you need an inexpensive place to stay for a...,0,bjTcT8Ty4cJZhEOEo01FGA
2,uYHaNptLzDLoV_JZ_MuzUA,0,2015-09-17,0,ne5WhI1jUFOcRn-b-gAzHA,3,Mittlerweile gibt es in Edinburgh zwei Ableger...,0,AXgRULmWcME7J6Ix3I--ww
3,uYHaNptLzDLoV_JZ_MuzUA,0,2016-08-21,0,llmdwOgDReucVoWEry61Lw,4,Location is everything and this hotel has it! ...,0,oU2SSOmsp_A8JYI7Z2JJ5w
4,uYHaNptLzDLoV_JZ_MuzUA,0,2013-11-20,0,DuffS87NaSMDmIfluvT83g,5,gute lage im stadtzentrum. shoppingmeile und s...,0,0xtbPEna2Kei11vsU-U2Mw


In [19]:
review.useful.describe()

count    4.736897e+06
mean     9.882921e-01
std      2.600021e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.000000e+00
max      1.125000e+03
Name: useful, dtype: float64

In [20]:
review[review['useful'] > 0]['useful'].describe()

count    1.981990e+06
mean     2.361989e+00
std      3.593301e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      3.000000e+00
max      1.125000e+03
Name: useful, dtype: float64

In [21]:
_ = (review[review['useful'] > 500])

_

,business_id,cool,date,funny,review_id,user_stars,review_txt,useful,user_id
4614802,DN0b4Un8--Uf6SEWLeh0UA,513,2010-08-01,631,A8mLBytNM2zmjHgSpsuZDA,1,"In retrospect, I should have known better than...",1125,qiTy11I-yp6foxIghRfGOA
4615135,DN0b4Un8--Uf6SEWLeh0UA,102,2012-11-18,262,69suGHxR3PVxicAgmUlhzA,1,We were very disappointed with this restaurant...,580,XmIsBXpdBevTYCe8gKQruQ


In [22]:
review.iloc[4614802,:]['review_txt']

'In retrospect, I should have known better than to step foot into Amy\'s Baking Company (a.k.a. ABC Bistro).  \n\n-8 pm on a Saturday night, three tables are occupied\n-a sign on the door indicates they\'re looking for an experienced line cook, a dishwasher, waitresses, you name it\n-Pita Jungle, just a stone\'s throw away, is packed to the gills\n\nThese are all bad signs.  They go ignored, however, because I\'d eaten one thing all day, and I had drove here to try their pizza.  Never, ever again.\n\nThe waitress brings me out a water and a menu to my table for one  on the patio, which I had all to myself.  Browsing the apps, I was pleasantly surprised to see several things priced $3, which seemed like a bargain, given the high price of everything else.  $3 tapanade sounded like a good deal, but I was informed that it didn\'t come with anything - it was $3 extra for bread.  Since I was solo, I decided to skip it and instead pay $14 for their 12" margherita pizza.  \n\nAbout three minut

In [23]:
business.review_count.describe()

count    156639.000000
mean         30.238159
std          96.486631
min           3.000000
25%           4.000000
50%           9.000000
75%          23.000000
max        6979.000000
Name: review_count, dtype: float64

In [24]:
business.city.value_counts(ascending=False)[:20]

Las Vegas      24768
Phoenix        15656
Toronto        15483
Charlotte       7557
Scottsdale      7510
Pittsburgh      5688
Montréal        5175
Mesa            5146
Henderson       4130
Tempe           3949
Chandler        3649
Edinburgh       3625
Cleveland       2979
Madison         2891
Glendale        2841
Gilbert         2835
Mississauga     2316
Stuttgart       1975
Peoria          1527
Markham         1390
Name: city, dtype: int64

In [25]:
business[business['business_id'] == 'DN0b4Un8--Uf6SEWLeh0UA']

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
59560,"7366 E Shea Blvd, Ste 112","{'GoodForMeal': {'dessert': True, 'latenight':...",DN0b4Un8--Uf6SEWLeh0UA,"[American (New), Italian, Food, Restaurants, P...",Scottsdale,"{'Tuesday': '12:00-21:00', 'Friday': '12:00-21...",0,33.584012,-111.922265,Amy's Baking Company,,85260,479,3.0,AZ


In [26]:
# Keep only Major Cities

business_maj = business.loc[business['city']\
                            .isin(['Phoenix', 'Charlotte', 'Toronto', 'Charlotte', 'Scottsdale'])]

In [27]:
business_maj.shape

(46206, 15)

In [28]:
## Analyze specific types of businesses

bus_small = business_maj[["name", "business_id", "categories", "attributes", 'stars', 'review_count', 'city', 'state', 'postal_code', 'latitude', 'longitude']].copy()
bus_small['categories2'] = bus_small['categories'].apply(lambda x: ', '.join(x)).str.replace(',', '')


In [29]:
bus_small.head()

,name,business_id,categories,attributes,stars,review_count,city,state,postal_code,latitude,longitude,categories2
1,South Florida Style Chicken & Ribs,mLwM-h2YhXl2NCgdS84_Bw,"[Food, Soul Food, Convenience Stores, Restaura...","{'GoodForMeal': {'dessert': False, 'latenight'...",4.5,4,Charlotte,NC,28215,35.236870,-80.741976,Food Soul Food Convenience Stores Restaurants
2,The Tea Emporium,v2WhjAB3PIBA8J8VxG3wEg,"[Food, Coffee & Tea]","{'BusinessParking': {'garage': False, 'street'...",4.5,7,Toronto,ON,M4K 1N7,43.677126,-79.353285,Food Coffee & Tea
3,TRUmatch,CVtCbSB1zUcUWg-9TNGTuQ,"[Professional Services, Matchmakers]",{},3.0,3,Scottsdale,AZ,85258,33.565082,-111.916400,Professional Services Matchmakers
4,Blimpie,duHFBe87uNSXImQmvBh87Q,"[Sandwiches, Restaurants]","{'RestaurantsTableService': False, 'GoodForMea...",4.5,10,Phoenix,AZ,85016,33.505928,-112.038847,Sandwiches Restaurants
5,Baxter's Cigars,uUEMrhJiL1a1pCA_I1SU7Q,"[Shopping, Tobacco Shops]","{'BusinessAcceptsCreditCards': True, 'Restaura...",5.0,35,Phoenix,AZ,85016,33.508068,-112.037552,Shopping Tobacco Shops


In [30]:
#Add Count of Name Frequency, identify potential chains
bus_small['name_cnt'] = bus_small['name'].groupby(bus_small['name']).transform('count')

In [31]:
# convert several Dummy Flags if category contains this description
for category in ['Sushi', 'Pizza', 'Restaurants']:
    bus_small.loc[:, category] = bus_small.categories2.str.contains(category).astype(int)

bus_small['Cat_Flag'] = (bus_small.Sushi + bus_small.Pizza > 0).astype(int)


In [32]:
# conditions = [
#     (df['Set'] == 'Z') & (df['Type'] == 'A'),
#     (df['Set'] == 'Z') & (df['Type'] == 'B'),
#     (df['Type'] == 'B')]
# choices = ['yellow', 'blue', 'purple']
# df['color'] = np.select(conditions, choices, default='black')

In [33]:
pd.crosstab(bus_small.Sushi, bus_small.Pizza)

Pizza,0,1
Sushi,,
0,44398,1254
1,550,4


In [34]:
# pd.crosstab(bus_small.Sushi, bus_small.Coffee)

In [35]:
#pd.crosstab(bus_small.Pizza, bus_small.Burger)

In [36]:
# pd.crosstab(bus_small.Pizza, bus_small.Coffee)

In [37]:
# bus_small.Coffee.value_counts()

In [38]:
bus_small.Pizza.value_counts()

0    44948
1     1258
Name: Pizza, dtype: int64

In [39]:
bus_small.Sushi.value_counts()

0    45652
1      554
Name: Sushi, dtype: int64

In [40]:
bus_small.Cat_Flag.value_counts()

0    44398
1     1808
Name: Cat_Flag, dtype: int64

#### Filter on review Count threshold

In [41]:
# review count threshold!!!
review_cnt = 10

#bus_list = bus_small[(bus_small['Cat_Flag'] == 1) & (bus_small['review_count'] >= review_cnt)]
bus_list = bus_small[(bus_small['Restaurants'] == 1) & 
                     (bus_small['review_count'] >= review_cnt) ]

In [42]:
bus_list.shape

(10100, 17)

#### Merge Business and Review Data


In [43]:
# Keep only recent reviews
rev_recent = review[review['date'] < '2017-01-01']

In [44]:
# Merge Businesses and Review
bus_review = bus_list.set_index('business_id').join(rev_recent.set_index('business_id'), how='left')
bus_review.reset_index(inplace = True)

In [47]:
# Append Reviewer history data
bus_review_elite = bus_review.set_index('user_id').join(user_small.set_index('user_id'), how='left')

bus_review_elite.reset_index(inplace = True)

In [48]:
bus_review_elite.head()

,user_id,business_id,name,categories,attributes,stars,review_count,city,state,postal_code,...,Cat_Flag,cool,date,funny,review_id,user_stars,review_txt,useful,yelping_since,first_yr_elite
0,---PLwSf5gKdIoVnyRHgBA,Y5CTA3S9TVkpr3Yvwaki-A,iPic Theaters,"[Bars, Arts & Entertainment, Nightlife, Cinema...",{'GoodForKids': True},4.0,473,Scottsdale,AZ,85254,...,0,0.0,2016-06-12,0.0,QXzgdKOdmrccxZ60Er7gig,3,I love iPic in that I don't like crowds and I ...,0.0,2015-07-31,99999.0
1,---cu1hq55BP9DWVXXKHZg,EGzsR_Tyd0QOaZ6g7wPl-w,Filibertos Mexican Food,"[Mexican, Restaurants]","{'RestaurantsTableService': False, 'GoodForMea...",4.5,10,Phoenix,AZ,85020,...,0,0.0,2014-11-19,0.0,C64IMmOZmJWiNdFe3ELklQ,4,Went to this location on 11/19/2014. Its not t...,1.0,2009-04-18,99999.0
2,---cu1hq55BP9DWVXXKHZg,eCJDiRPoSuA0bsb7F1gNUA,Asian Cafe,"[Asian Fusion, Chinese, Restaurants, Fast Food...","{'GoodForMeal': {'dessert': False, 'latenight'...",4.0,48,Phoenix,AZ,85023,...,0,0.0,2014-11-20,0.0,O0frjb1NA2AD5VnHNWVpcg,1,"Somehow 11111 N 7th Street is ""too far for us....",0.0,2009-04-18,99999.0
3,--2HUmLkcNHZp0xw6AMBPg,Hpi9raHlFm6prTSzFHTurg,Corleones Philly Steaks,"[Chicken Wings, Sandwiches, Restaurants, Pizza...","{'Alcohol': 'none', 'HasTV': True, 'NoiseLevel...",3.5,124,Scottsdale,AZ,85260,...,1,0.0,2016-07-29,0.0,PX83ZIZisbi6IFN9Guk9ZQ,5,Great team! Love this place. I have come in r...,0.0,2016-07-28,99999.0
4,--3WaS23LcIXtxyFULJHTA,ADgacmZ-qXrSOhMfU6bmTA,5Church,"[Food, Restaurants, American (New), Breakfast ...","{'Alcohol': 'full_bar', 'HasTV': True, 'NoiseL...",3.5,507,Charlotte,NC,28202,...,0,1.0,2016-02-08,0.0,rsYI3LPC6a7JhjjDYNCTKw,3,We stopped in here for drinks and appetizers o...,1.0,2010-05-02,2012.0


#### convert to date and create categorical vars

In [49]:
# convert to date
bus_review_elite.date = pd.to_datetime(bus_review_elite.date, format="%Y/%m/%d")

# convert some vars to category
for col in ['city', 'state', 'postal_code', 'stars']:
    bus_review_elite[col] = bus_review_elite[col].astype('category')


In [50]:
bus_review_elite['elite_prior'] = (bus_review_elite['first_yr_elite'] <= bus_review_elite['date'].dt.year).astype(int)


In [51]:
bus_review_elite.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750350 entries, 0 to 750349
Data columns (total 28 columns):
user_id           750124 non-null object
business_id       750350 non-null object
name              750350 non-null object
categories        750350 non-null object
attributes        750350 non-null object
stars             750350 non-null category
review_count      750350 non-null int64
city              750350 non-null category
state             750350 non-null category
postal_code       750350 non-null category
latitude          750350 non-null float64
longitude         750350 non-null float64
categories2       750350 non-null object
name_cnt          750350 non-null int64
Sushi             750350 non-null int64
Pizza             750350 non-null int64
Restaurants       750350 non-null int64
Cat_Flag          750350 non-null int64
cool              750124 non-null float64
date              750124 non-null datetime64[ns]
funny             750124 non-null float64
review_id     

#### pickle bus + review (big) file

In [52]:
bus_review_elite.to_pickle('data/bus_review_elite.pkl')

In [53]:
bus_review_elite.shape

(750350, 28)

In [54]:
bus_review_elite.head()

,user_id,business_id,name,categories,attributes,stars,review_count,city,state,postal_code,...,cool,date,funny,review_id,user_stars,review_txt,useful,yelping_since,first_yr_elite,elite_prior
0,---PLwSf5gKdIoVnyRHgBA,Y5CTA3S9TVkpr3Yvwaki-A,iPic Theaters,"[Bars, Arts & Entertainment, Nightlife, Cinema...",{'GoodForKids': True},4.0,473,Scottsdale,AZ,85254,...,0.0,2016-06-12,0.0,QXzgdKOdmrccxZ60Er7gig,3,I love iPic in that I don't like crowds and I ...,0.0,2015-07-31,99999.0,0
1,---cu1hq55BP9DWVXXKHZg,EGzsR_Tyd0QOaZ6g7wPl-w,Filibertos Mexican Food,"[Mexican, Restaurants]","{'RestaurantsTableService': False, 'GoodForMea...",4.5,10,Phoenix,AZ,85020,...,0.0,2014-11-19,0.0,C64IMmOZmJWiNdFe3ELklQ,4,Went to this location on 11/19/2014. Its not t...,1.0,2009-04-18,99999.0,0
2,---cu1hq55BP9DWVXXKHZg,eCJDiRPoSuA0bsb7F1gNUA,Asian Cafe,"[Asian Fusion, Chinese, Restaurants, Fast Food...","{'GoodForMeal': {'dessert': False, 'latenight'...",4.0,48,Phoenix,AZ,85023,...,0.0,2014-11-20,0.0,O0frjb1NA2AD5VnHNWVpcg,1,"Somehow 11111 N 7th Street is ""too far for us....",0.0,2009-04-18,99999.0,0
3,--2HUmLkcNHZp0xw6AMBPg,Hpi9raHlFm6prTSzFHTurg,Corleones Philly Steaks,"[Chicken Wings, Sandwiches, Restaurants, Pizza...","{'Alcohol': 'none', 'HasTV': True, 'NoiseLevel...",3.5,124,Scottsdale,AZ,85260,...,0.0,2016-07-29,0.0,PX83ZIZisbi6IFN9Guk9ZQ,5,Great team! Love this place. I have come in r...,0.0,2016-07-28,99999.0,0
4,--3WaS23LcIXtxyFULJHTA,ADgacmZ-qXrSOhMfU6bmTA,5Church,"[Food, Restaurants, American (New), Breakfast ...","{'Alcohol': 'full_bar', 'HasTV': True, 'NoiseL...",3.5,507,Charlotte,NC,28202,...,1.0,2016-02-08,0.0,rsYI3LPC6a7JhjjDYNCTKw,3,We stopped in here for drinks and appetizers o...,1.0,2010-05-02,2012.0,1


In [55]:
bus_review.city.value_counts()

Phoenix       258718
Toronto       230906
Scottsdale    142844
Charlotte     117882
Name: city, dtype: int64

In [4]:
bus_review_elite = np.load('data/bus_review_elite.pkl')

### Compute User Review Count

In [12]:
user_rvw_cnt = bus_review_elite.groupby('user_id').count().reset_index()[['user_id', 'state']]

In [14]:
user_rvw_cnt.columns = ['user_id', 'user_rvw_cnt']
user_rvw_cnt.head()
# [['review_id', 'state']]

,user_id,user_rvw_cnt
0,---PLwSf5gKdIoVnyRHgBA,1
1,---cu1hq55BP9DWVXXKHZg,2
2,--2HUmLkcNHZp0xw6AMBPg,1
3,--3WaS23LcIXtxyFULJHTA,7
4,--41c9Tl0C9OGewIR7Qyzg,1


In [31]:
pd.cut(user_rvw_cnt.user_rvw_cnt, 700).value_counts()

(0.304, 1.994]        127341
(1.994, 2.989]         37512
(2.989, 3.983]         17701
(3.983, 4.977]         10333
(4.977, 5.971]          6434
(5.971, 6.966]          4664
(6.966, 7.96]           3272
(7.96, 8.954]           2532
(8.954, 9.949]          1940
(9.949, 10.943]         1524
(10.943, 11.937]        1231
(11.937, 12.931]        1000
(12.931, 13.926]         849
(13.926, 14.92]          732
(14.92, 15.914]          641
(15.914, 16.909]         551
(16.909, 17.903]         478
(18.897, 19.891]         406
(17.903, 18.897]         397
(19.891, 20.886]         351
(20.886, 21.88]          293
(21.88, 22.874]          248
(22.874, 23.869]         241
(23.869, 24.863]         214
(24.863, 25.857]         187
(26.851, 27.846]         182
(25.857, 26.851]         181
(27.846, 28.84]          135
(30.829, 31.823]         135
(28.84, 29.834]          130
                       ...  
(319.171, 320.166]         0
(317.183, 318.177]         0
(290.337, 291.331]         0
(316.189, 317.

In [34]:
# Keep those that had 3 or more reviews
user_rvw_cnt_mny = user_rvw_cnt[user_rvw_cnt['user_rvw_cnt'] >= 3]

In [35]:
print(user_rvw_cnt.shape)
print(user_rvw_cnt_mny.shape)

(224576, 2)
(59723, 2)


In [36]:
# This Df is at the review level
user_rvw_cnt_mny_df = pd.merge(user_rvw_cnt, bus_review_elite, how='inner', left_on = 'user_id', right_on = 'user_id')
user_rvw_cnt_mny_df.shape

(750124, 29)

#### randomly subsample reviews

In [56]:
bus_review_small = bus_review_elite.sample(n = 50000, random_state=50)
bus_review_small.drop_duplicates(['review_id'], keep='first', inplace=True)

In [57]:
bus_review_small.to_pickle('data/bus_review_small.pkl')

In [58]:
bus_review.loc[bus_review['useful'] > 1000, 'review_txt']

173465    In retrospect, I should have known better than...
Name: review_txt, dtype: object

In [52]:
#bus_review.set_index('review_id', inplace=True)

#### Randomly subsample from those that had many reviews (3 or more)

In [37]:
bus_review_small_mny = user_rvw_cnt_mny_df.sample(n = 50000, random_state=50)
bus_review_small_mny.drop_duplicates(['review_id'], keep='first', inplace=True)

In [38]:
bus_review_small_mny.to_pickle('data/bus_review_small_mny.pkl')

In [39]:
bus_review_small_mny.columns

Index(['user_id', 'user_rvw_cnt', 'business_id', 'name', 'categories',
       'attributes', 'stars', 'review_count', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'categories2', 'name_cnt', 'Sushi', 'Pizza',
       'Restaurants', 'Cat_Flag', 'cool', 'date', 'funny', 'review_id',
       'user_stars', 'review_txt', 'useful', 'yelping_since', 'first_yr_elite',
       'elite_prior'],
      dtype='object')

### Sentence Tokenize

In [5]:
bus_review_small = np.load('data/bus_review_small.pkl')

In [292]:
bus_review_small.iloc[:10,:]['review_txt']

544639    I was thrilled when I saw on twitter that anot...
282611    went here on a break from Jury Duty once. it w...
352137    Despite the reviews about slow waiting times w...
463957    Food quality a 10. Had the pulled pork with ch...
683036    Great drinks; spectacular sweet potato fries, ...
626109    Out of the way place that serves fresh flavorf...
246475    Came here with the husband for dinner and orde...
542830    I was immediately greeted with friendly faces....
653769    This place is so awesome that I literally can'...
560460    As many of the reviewers, I heard about Over E...
Name: review_txt, dtype: object

In [293]:
def custom_tokenize(text):
    try:
        return sent_tokenize(text)
    except:
        return ''


In [294]:
bus_review_small.shape

(50000, 28)

In [295]:
bus_review_small['sent_token'] = bus_review_small['review_txt'].apply(custom_tokenize) 

In [9]:
bus_review_small.head()

,business_id,name,categories,attributes,stars,review_count,city,state,postal_code,latitude,...,Cat_Flag,cool,date,funny,review_id,user_stars,review_txt,useful,user_id,sent_token
544639,iGEvDk6hsizigmXhDKs2Vg,Seven Lives Tacos Y Mariscos,"[Restaurants, Mexican, Seafood]","{'RestaurantsTableService': False, 'GoodForMea...",4.5,838,Toronto,ON,M5T 2K2,43.654341,...,0,0.0,2016-02-12,0.0,i-QGnPlonj9KGSgfSKSL-A,5,The best seafood tacos in the city. A little p...,0.0,Lk7gBKmQ9qQnEioIhB0-yQ,"[The best seafood tacos in the city., A little..."
282611,NFm869_w6cvVaWaNpAzjeA,Soul Gastrolounge,"[Nightlife, Tapas Bars, Lounges, Restaurants, ...","{'Alcohol': 'full_bar', 'HasTV': False, 'Noise...",4.0,764,Charlotte,NC,28205,35.220214,...,0,0.0,2014-03-27,0.0,2yDbp3hxbDyOuQV-bIRN6w,5,This place is so much fun to go to!!! I am in...,0.0,zGD1LkmZLX2FrVbj0iTtzA,"[This place is so much fun to go to!!!, I am i..."
352137,T6ha3lAVg7mjr6GTnH9V-A,Pojangmacha Food,"[Food Stands, Restaurants, Korean, Fast Food]","{'RestaurantsTableService': False, 'GoodForMea...",4.0,74,Toronto,ON,M2N 5S2,43.779100,...,0,0.0,2012-01-21,0.0,Y_-GZJTe99tVrVhL1St9vA,4,One of the fave street food for people in Nort...,0.0,Upq-i76O879cJIb6DbrpiA,[One of the fave street food for people in Nor...
463957,b_8jCti0vuouJ_fjZpdmZw,Soul Cafe,"[Breakfast & Brunch, Music Venues, Nightlife, ...","{'Alcohol': 'full_bar', 'HasTV': True, 'NoiseL...",4.0,315,Scottsdale,AZ,85255,33.698164,...,0,0.0,2015-11-02,0.0,azZ1NH_Wy7hTQOjevs7TcA,5,We brought our friends from New Jersey here fo...,0.0,tNj_oJiHZPSInVFOGnRyFg,[We brought our friends from New Jersey here f...
683036,u6xb2aUAvX6i1WhsQWqgWg,Lo-Lo's Chicken & Waffles,"[Breakfast & Brunch, Southern, Soul Food, Rest...","{'RestaurantsTableService': True, 'GoodForMeal...",4.0,620,Scottsdale,AZ,85257,33.478732,...,0,1.0,2012-08-11,0.0,aFA-jzA3pHz0JXK_ZpN0eQ,4,"Oh thank you, crispy waffles and chicken... Se...",0.0,CZLCeIDYQiGJSquzg73jMQ,"[Oh thank you, crispy waffles and chicken... S..."


In [296]:
# Each row is a sentence
review_sent = pd.DataFrame(bus_review_small.set_index(['review_id'])['sent_token'].apply(pd.Series).stack()).reset_index()

In [297]:
review_sent.head()

,review_id,level_1,0
0,DBFkxzPlSx0VBLdCGWPXpA,0,I was thrilled when I saw on twitter that anot...
1,DBFkxzPlSx0VBLdCGWPXpA,1,They are officially open today!
2,DBFkxzPlSx0VBLdCGWPXpA,2,"Located at 261 Queen W, in my opinion this is ..."
3,DBFkxzPlSx0VBLdCGWPXpA,3,Who wouldn't want to have a comforting hot bow...
4,DBFkxzPlSx0VBLdCGWPXpA,4,Can you believe that they have 450 Arita porce...


In [298]:
review_sent['sent_cnt'] = review_sent['review_id'].groupby(review_sent['review_id']).transform('count')
review_sent.drop('level_1', axis=1, inplace=True)

review_sent.columns = ['review_id', 'sentence', 'sent_cnt']


In [299]:
review_sent.shape

(428381, 3)

In [300]:
review_sent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428381 entries, 0 to 428380
Data columns (total 3 columns):
review_id    428363 non-null object
sentence     428381 non-null object
sent_cnt     428363 non-null float64
dtypes: float64(1), object(2)
memory usage: 9.8+ MB


In [301]:
#bus_review_small.duplicated('review_id')
names = bus_review_small.review_id.value_counts()
names[names > 1]

Series([], Name: review_id, dtype: int64)

In [302]:
bus_review_small.review_id.shape

(50000,)

In [303]:
### Drop duplicates review Ids

In [304]:
bus_review_small.drop_duplicates(['review_id'], keep='first', inplace=True)

In [305]:
review_sent_merge = pd.merge(bus_review_small, review_sent, how='inner', left_on = 'review_id', right_on = 'review_id')


In [306]:
review_sent_merge.head(10)

,user_id,business_id,name,categories,attributes,stars,review_count,city,state,postal_code,...,review_id,user_stars,review_txt,useful,yelping_since,first_yr_elite,elite_prior,sent_token,sentence,sent_cnt
0,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,DBFkxzPlSx0VBLdCGWPXpA,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,I was thrilled when I saw on twitter that anot...,27.0
1,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,DBFkxzPlSx0VBLdCGWPXpA,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,They are officially open today!,27.0
2,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,DBFkxzPlSx0VBLdCGWPXpA,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,"Located at 261 Queen W, in my opinion this is ...",27.0
3,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,DBFkxzPlSx0VBLdCGWPXpA,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,Who wouldn't want to have a comforting hot bow...,27.0
4,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,DBFkxzPlSx0VBLdCGWPXpA,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,Can you believe that they have 450 Arita porce...,27.0
5,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,DBFkxzPlSx0VBLdCGWPXpA,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,So it is not surprising that they even importe...,27.0
6,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,DBFkxzPlSx0VBLdCGWPXpA,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,"Unlike most other ramen shops, Touhenboku used...",27.0
7,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,DBFkxzPlSx0VBLdCGWPXpA,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,Chef Machida also mixed his own flour blend in...,27.0
8,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,DBFkxzPlSx0VBLdCGWPXpA,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,There's 4 types of ramen on their menu : White...,27.0
9,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restauran

In [307]:
# prog = re.compile(pattern)
# result = prog.match(string)

In [308]:
# prog = re.compile('^([a-zA-Z]+|\W)$')

#### Tokenize

In [309]:
def tokenize_stem(df, col_name):
    '''
    Input dataframe.
    Output dataframe do stemming on a specific column
    '''
    # Tokenize
    df['token'] = df[col_name].apply(word_tokenize) 

    # Create stem words
    prog = re.compile('^([a-zA-Z]+|\W)$')
    
#     df['stemmed'] = df["token"].\
#         apply(lambda x: [stemmer.stem(y) for y in x if re.match('^([a-zA-Z]+|\W)$', y)])
    df['stemmed'] = df["token"].\
        apply(lambda x: [stemmer.stem(y) for y in x if prog.match(y)])

    df['stemmed2'] = df['stemmed'].apply(lambda x: ', '.join(x)).str.replace(',', '')
#   df['stemmed2'] = df['stemmed2'].str.replace(',', '')
        
    df.drop('token', axis=1)
    df.drop('stemmed', axis=1)

    return df

In [310]:
### Create a small file
review_sent_merge.shape

(428381, 31)

In [311]:
review_sent_merge.head()

,user_id,business_id,name,categories,attributes,stars,review_count,city,state,postal_code,...,review_id,user_stars,review_txt,useful,yelping_since,first_yr_elite,elite_prior,sent_token,sentence,sent_cnt
0,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,DBFkxzPlSx0VBLdCGWPXpA,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,I was thrilled when I saw on twitter that anot...,27.0
1,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,DBFkxzPlSx0VBLdCGWPXpA,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,They are officially open today!,27.0
2,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,DBFkxzPlSx0VBLdCGWPXpA,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,"Located at 261 Queen W, in my opinion this is ...",27.0
3,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,DBFkxzPlSx0VBLdCGWPXpA,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,Who wouldn't want to have a comforting hot bow...,27.0
4,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,DBFkxzPlSx0VBLdCGWPXpA,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,Can you believe that they have 450 Arita porce...,27.0


### Sentiment Analysis - sentence at a time

In [312]:
# bus_review_small['sent_token'] = bus_review_small['review_txt'].apply(custom_tokenize) 

def custom_sent_pol(text):
    try:
        return TextBlob(text).sentiment[0]
    except:
        return 0
    



In [313]:
# Sentence level sentiment
review_sent_merge['polarity'] = review_sent_merge['sentence'].apply(custom_sent_pol)


In [314]:
review_sent_merge.shape

(428381, 32)

In [315]:
review_sent_merge.head()

,user_id,business_id,name,categories,attributes,stars,review_count,city,state,postal_code,...,user_stars,review_txt,useful,yelping_since,first_yr_elite,elite_prior,sent_token,sentence,sent_cnt,polarity
0,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,I was thrilled when I saw on twitter that anot...,27.0,0.425
1,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,They are officially open today!,27.0,0.000
2,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,"Located at 261 Queen W, in my opinion this is ...",27.0,1.000
3,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,Who wouldn't want to have a comforting hot bow...,27.0,0.125
4,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,Can you believe that they have 450 Arita porce...,27.0,0.250


#### Tokenize by sentence

In [316]:
reviews_sent_token = tokenize_stem(review_sent_merge, 'sentence')

In [317]:
reviews_sent_token.head()

,user_id,business_id,name,categories,attributes,stars,review_count,city,state,postal_code,...,yelping_since,first_yr_elite,elite_prior,sent_token,sentence,sent_cnt,polarity,token,stemmed,stemmed2
0,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,I was thrilled when I saw on twitter that anot...,27.0,0.425,"[I, was, thrilled, when, I, saw, on, twitter, ...","[i, was, thrill, when, i, saw, on, twitter, th...",i was thrill when i saw on twitter that anoth ...
1,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,They are officially open today!,27.0,0.000,"[They, are, officially, open, today, !]","[they, are, offici, open, today, !]",they are offici open today !
2,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,"Located at 261 Queen W, in my opinion this is ...",27.0,1.000,"[Located, at, 261, Queen, W, ,, in, my, opinio...","[locat, at, queen, w, ,, in, my, opinion, this...",locat at queen w in my opinion this is probab...
3,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,Who wouldn't want to have a comforting hot bow...,27.0,0.125,"[Who, would, n't, want, to, have, a, comfortin...","[who, would, want, to, have, a, comfort, hot, ...",who would want to have a comfort hot bowl of r...
4,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,Can you believe that they have 450 Arita porce...,27.0,0.250,"[Can, you, believe, that, they, have, 450, Ari...","[can, you, believ, that, they, have, arita, po...",can you believ that they have arita porcelain ...


In [318]:
reviews_sent_token.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 428381 entries, 0 to 428380
Data columns (total 35 columns):
user_id           428363 non-null object
business_id       428381 non-null object
name              428381 non-null object
categories        428381 non-null object
attributes        428381 non-null object
stars             428381 non-null category
review_count      428381 non-null int64
city              428381 non-null category
state             428381 non-null category
postal_code       428381 non-null category
latitude          428381 non-null float64
longitude         428381 non-null float64
categories2       428381 non-null object
name_cnt          428381 non-null int64
Sushi             428381 non-null int64
Pizza             428381 non-null int64
Restaurants       428381 non-null int64
Cat_Flag          428381 non-null int64
cool              428363 non-null float64
date              428363 non-null datetime64[ns]
funny             428363 non-null float64
review_id     

In [319]:
reviews_sent_token.shape

(428381, 35)

In [320]:
reviews_sent_token.to_pickle('data/reviews_sent_token.pkl')

### Load dataset

In [321]:
reviews_sent_token = np.load('data/reviews_sent_token.pkl')

In [322]:
(reviews_sent_token.memory_usage(deep=True).sum())/(10e9)

0.15265344310000001

In [323]:
#reviews_tokenized = tokenize_stem(reviews_token, 'review_txt')

In [324]:
#reviews_token.shape

In [325]:
#reviews_tokenized.to_pickle('data/reviews_tokenized.pkl')
reviews_sent_token.shape

(428381, 35)

In [326]:
reviews_sent_token.head()

,user_id,business_id,name,categories,attributes,stars,review_count,city,state,postal_code,...,yelping_since,first_yr_elite,elite_prior,sent_token,sentence,sent_cnt,polarity,token,stemmed,stemmed2
0,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,I was thrilled when I saw on twitter that anot...,27.0,0.425,"[I, was, thrilled, when, I, saw, on, twitter, ...","[i, was, thrill, when, i, saw, on, twitter, th...",i was thrill when i saw on twitter that anoth ...
1,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,They are officially open today!,27.0,0.000,"[They, are, officially, open, today, !]","[they, are, offici, open, today, !]",they are offici open today !
2,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,"Located at 261 Queen W, in my opinion this is ...",27.0,1.000,"[Located, at, 261, Queen, W, ,, in, my, opinio...","[locat, at, queen, w, ,, in, my, opinion, this...",locat at queen w in my opinion this is probab...
3,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,Who wouldn't want to have a comforting hot bow...,27.0,0.125,"[Who, would, n't, want, to, have, a, comfortin...","[who, would, want, to, have, a, comfort, hot, ...",who would want to have a comfort hot bowl of r...
4,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...,Can you believe that they have 450 Arita porce...,27.0,0.250,"[Can, you, believe, that, they, have, 450, Ari...","[can, you, believ, that, they, have, arita, po...",can you believ that they have arita porcelain ...


#### Count Vectorize and LDA on all Sentences

In [327]:
# reviews_vect_df['review_txt'].head()[0]

In [328]:
#reviews_sent_token.head()

In [329]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print( "Topic %d:" % (topic_idx + 1))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[ :-no_top_words - 1:-1]]))


In [330]:
#lda_fit.components_

In [331]:
no_topics = 30
no_top_words = 10

LDA_cols = ['LDA_' + str(i) for i in range(1, no_topics + 1)]


In [388]:
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.90, min_df=20, stop_words = my_stop_words)

tf = tf_vectorizer.fit_transform(reviews_sent_token['stemmed2'])
tf_feature_names = tf_vectorizer.get_feature_names()


In [389]:
lda = LatentDirichletAllocation(n_components = no_topics, max_iter = 10, learning_method='online',
                                learning_offset=50., random_state = 10)




In [390]:
lda_fit = lda.fit(tf)

In [335]:
# save the model to disk
# https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
filename = 'lda.sav'
pickle.dump(lda, open(filename, 'wb'))


filename = 'lda_fit.sav'
pickle.dump(lda_fit, open(filename, 'wb'))


### NMF

In [381]:
more_words = ['pizza', 'beer', 'burger', 'salad', 'mexican', 'chicken', 'phoenix', 'fri', 'taco', 'toronto', 'charlott', 'sauc']

my_stop_words = text.ENGLISH_STOP_WORDS.union(more_words)

In [384]:
# TFIDF
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=20, stop_words = my_stop_words)
# stop_words='english'
tfidf = tfidf_vectorizer.fit_transform(reviews_sent_token['stemmed2'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()


In [385]:
nmf = NMF(n_components = 20, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)


In [387]:
display_topics(nmf, tfidf_feature_names, no_top_words=10)

Topic 1:
great atmospher experi price select spot overal tast lunch drink
Topic 2:
good pretti price tast overal select thing look alway portion
Topic 3:
place recommend awesom favorit look high new want star busi
Topic 4:
food price excel qualiti fresh fast decent awesom fantast ok
Topic 5:
like tast look feel felt someth did fresh lot chees
Topic 6:
servic excel custom slow fast quick alway terribl bad attent
Topic 7:
love absolut atmospher decor husband everyth locat patio ambianc restaur
Topic 8:
veri nice tasti disappoint price fresh attent reason flavor atmospher
Topic 9:
time everi went long second visit sever mani lunch alway
Topic 10:
order drink dish chees took sandwich came minut appet got
Topic 11:
definit recommend return high worth visit area soon check anyon
Topic 12:
delici fresh everyth absolut perfect flavor alway sandwich tast bread
Topic 13:
realli enjoy nice want tasti impress cool experi bad atmospher
Topic 14:
friend staff alway super attent help server recommend

### LDA Topics

In [393]:
#display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda_fit, tf_feature_names, no_top_words=15)

Topic 1:
sandwich special ca noth option decent stop food absolut pack consist typic present lettuc cafe
Topic 2:
price recommend chees star day bad expect high els wing bbq street averag cheap money
Topic 3:
realli flavor work beef fish grill week surpris veggi help wife extrem pleas complet pleasant
Topic 4:
good food definit pretti disappoint qualiti left gave salmon ambianc varieti expens given receiv standard
Topic 5:
came lunch small hour rice worth big chip bacon bean glass cocktail dure beauti space
Topic 6:
order dinner end flavour hand pepper comfort boyfriend basic ramen readi mouth min tortilla oyster
Topic 7:
dish breakfast waiter bring sat chef french butter turn saw rate greet unfortun pancak paid
Topic 8:
ask bread open return point free hope okay wish dress eaten season bun sign complaint
Topic 9:
food servic wait tabl nice minut excel walk wine patio outsid share second crispi pick
Topic 10:
got lot locat meat anoth sit probabl close let leav pasta oh list shop soon
T

In [337]:
# load the model from disk
filename = 'lda_fit.sav'

lda_fit = pickle.load(open(filename, 'rb'))


In [338]:
# Create a DF with LDA cols
lda_x = pd.DataFrame(lda.transform(tf), columns = LDA_cols)


### Pickle file

In [339]:
lda_x.to_pickle('data/lda_x.pkl')

In [340]:
lda_x.shape

(428381, 30)

In [341]:
lda_x.head(10)

,LDA_1,LDA_2,LDA_3,LDA_4,LDA_5,LDA_6,LDA_7,LDA_8,LDA_9,LDA_10,...,LDA_21,LDA_22,LDA_23,LDA_24,LDA_25,LDA_26,LDA_27,LDA_28,LDA_29,LDA_30
0,0.602222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,...,0.002222,0.002222,0.002222,0.002222,0.002222,0.135556,0.002222,0.002222,0.002222,0.002222
1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.258333,0.008333,...,0.008333,0.008333,0.258333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.103333,0.003333,0.103333,0.003333,...,0.003333,0.003333,0.003333,0.103333,0.003333,0.503333,0.003333,0.003333,0.003333,0.003333
3,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,...,0.002381,0.002381,0.002381,0.002381,0.002381,0.073810,0.002381,0.073810,0.073810,0.002381
4,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,...,0.002778,0.002778,0.002778,0.002778,0.002778,0.086111,0.002778,0.086111,0.002778,0.002778
5,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,...,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.006667,0.806667,0.006667,0.006667
6,0.073810,0.073810,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,...,0.002381,0.002381,0.288095,0.002381,0.073810,0.073810,0.145238,0.002381,0.002381,0.002381
7,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.086111,...,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.086111,0.002778,0.002778,0.586111
8,0.002083,0.002083,0.002083,0.002083,0.064583,0.002083,0.002083,0.002083,0.439583,0.189583,...,0.064583,0.002083,0.002083,0.002083,0.002083,0.064583,0.064583,0.002083,0.002083,0.002083
9,0.003333,0.103333,0.003333,0.003333,0.003333,0.803333,0.003333,0.003333,0.003333,0.003333,...,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333


In [342]:
# First merge back to original df
review_LDA = pd.merge(reviews_sent_token, lda_x, left_index = True, right_index = True)


In [343]:
review_LDA.columns

Index(['user_id', 'business_id', 'name', 'categories', 'attributes', 'stars',
       'review_count', 'city', 'state', 'postal_code', 'latitude', 'longitude',
       'categories2', 'name_cnt', 'Sushi', 'Pizza', 'Restaurants', 'Cat_Flag',
       'cool', 'date', 'funny', 'review_id', 'user_stars', 'review_txt',
       'useful', 'yelping_since', 'first_yr_elite', 'elite_prior',
       'sent_token', 'sentence', 'sent_cnt', 'polarity', 'token', 'stemmed',
       'stemmed2', 'LDA_1', 'LDA_2', 'LDA_3', 'LDA_4', 'LDA_5', 'LDA_6',
       'LDA_7', 'LDA_8', 'LDA_9', 'LDA_10', 'LDA_11', 'LDA_12', 'LDA_13',
       'LDA_14', 'LDA_15', 'LDA_16', 'LDA_17', 'LDA_18', 'LDA_19', 'LDA_20',
       'LDA_21', 'LDA_22', 'LDA_23', 'LDA_24', 'LDA_25', 'LDA_26', 'LDA_27',
       'LDA_28', 'LDA_29', 'LDA_30'],
      dtype='object')

In [344]:
review_LDA.shape

(428381, 65)

   ### Split LDA by Good / Bad Reviews

In [ ]:
reviews_sent_token = np.load('data/reviews_sent_token.pkl')

In [417]:
good_r = reviews_sent_token.loc[(reviews_sent_token['user_stars'] == 4) | (reviews_sent_token['user_stars'] == 5)]

bad_r = reviews_sent_token.loc[(reviews_sent_token['user_stars'] == 1) |
                               (reviews_sent_token['user_stars'] == 2) |
                               (reviews_sent_token['user_stars'] == 3)]

In [450]:
# small sample
good_r_small = good_r.iloc[0:1000]
bad_r_small = bad_r.iloc[0:1000]

In [457]:
no_topics = 10
no_top_words = 10

LDA_cols = ['LDA_' + str(i) for i in range(1, no_topics + 1)]

In [445]:
# Define vectorizer and LDA
tf_vectorizer = CountVectorizer(max_df=0.90, min_df=20, stop_words = my_stop_words)

# LDA
lda = LatentDirichletAllocation(n_components = no_topics, max_iter = 10, learning_method='online',
                                learning_offset=50., random_state = 10)


In [471]:
def Df_Vect(df, vectorizer, column):
    tf_vectorized = vectorizer.fit_transform(df[column])
    tf_feature_names = vectorizer.get_feature_names()
    return tf_vectorized, tf_feature_names

def LDA_fit(tf_vectorized, LDA_cols, tf_feature_names):
    lda_fit = lda.fit(tf_vectorized)
    
    # df with weights
    lda_x = pd.DataFrame(lda.transform(tf_vectorized), columns = LDA_cols)

    #display_topics(nmf, tfidf_feature_names, no_top_words)
    display_topics(lda_fit, tf_feature_names, no_top_words=15)
    
    return lda_fit, lda_x

# Merge LDA to main df and also return collapsed DF
def LDA_merge(big_df, main_df, lda_df, LDA_cols):
    #review_LDA = pd.merge(reviews_sent_token, lda_x, left_index = True, right_index = True)
    review_LDA = pd.merge(main_df, lda_df, left_index = True, right_index = True)
    
    review_LDA_agg = review_LDA.groupby('review_id').mean().reset_index()[['review_id'] + ['sent_cnt'] + LDA_cols]
    
    bus_review_LDA = big_df.set_index('review_id').join(review_LDA_agg.set_index('review_id'), how ='inner')
    bus_review_LDA.reset_index(inplace=True)
    return review_LDA, bus_review_LDA

In [472]:
good_r_small.review_id.nunique()

131

In [477]:
bus_review_small.head()

,user_id,business_id,name,categories,attributes,stars,review_count,city,state,postal_code,...,date,funny,review_id,user_stars,review_txt,useful,yelping_since,first_yr_elite,elite_prior,sent_token
544639,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,M5V 1Z4,...,2013-11-17,0.0,DBFkxzPlSx0VBLdCGWPXpA,3,I was thrilled when I saw on twitter that anot...,2.0,2010-11-10,2013.0,1,[I was thrilled when I saw on twitter that ano...
282611,NMmu4oC-YRqdi0WY9pV78g,1zH3_Agw8CTgSmKgg4JNmg,Yasda Bento,"[Korean, Noodles, Japanese, Restaurants]","{'RestaurantsTableService': False, 'GoodForMea...",3.5,61,Phoenix,AZ,85003,...,2008-08-22,1.0,SnwEJ7lAqGs6kZ2nbghudw,2,went here on a break from Jury Duty once. it w...,2.0,2008-08-18,2008.0,1,"[went here on a break from Jury Duty once., it..."
352137,T23h5oJqopYrLXp_A1GFYA,8XoDXN2DKmQGJHZhkQbJ9A,Emily's Cafe,"[Restaurants, Breakfast & Brunch, Cafes]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,79,Scottsdale,AZ,85255,...,2016-05-23,0.0,6GcGV7Sq-axwgmPBdEy3fQ,3,Despite the reviews about slow waiting times w...,0.0,2014-05-23,99999.0,0,[Despite the reviews about slow waiting times ...
463957,bK-lus15ReXTyDqdEj4OQg,6nKR80xEGHYf2UxAe_Cu_g,The Thumb BBQ,"[Gas Stations, Burgers, Automotive, American (...","{'Alcohol': 'beer_and_wine', 'HasTV': True, 'N...",4.0,660,Scottsdale,AZ,85260,...,2013-04-29,0.0,0VX2IuiyGmC-LrLfYUnGeQ,5,Food quality a 10. Had the pulled pork with ch...,0.0,2011-11-04,99999.0,0,"[Food quality a 10., Had the pulled pork with ..."
683036,uAf25oGZ8aURLI3u57GF_w,NPiLy71qqXzb45k9eQOgCQ,Maizie's Cafe & Bistro,"[Cafes, Restaurants, American (New)]","{'RestaurantsTableService': True, 'GoodForMeal...",4.0,382,Phoenix,AZ,85012,...,2011-04-18,0.0,Cd3JHCfGtdYdMJ2_Ipqh6Q,5,"Great drinks; spectacular sweet potato fries, ...",1.0,2011-04-16,99999.0,0,"[Great drinks; spectacular sweet potato fries,..."


In [475]:
good_r_small.shape

(1000, 35)

In [455]:
good_tf, good_feat = Df_Vect(good_r_small, tf_vectorizer, 'stemmed2')

In [458]:
lda_good, lda_x_good = LDA_fit(good_tf, LDA_cols, good_feat)

Topic 1:
tri delici realli drink love good nice fresh food time thing alway order servic amaz
Topic 2:
place servic littl staff friend veri onli becaus nice thing price food great tri recommend
Topic 3:
got dish wait did look thing come becaus nice good staff food great eat onli
Topic 4:
food good price look realli onli fresh ca drink like just great tabl place tri
Topic 5:
tabl ca wait nice food veri friend great place servic amaz love definit littl recommend
Topic 6:
sandwich menu friend onli good thing nice delici food alway amaz perfect like servic best
Topic 7:
order best food menu becaus sandwich ca price amaz thing friend staff tri come got
Topic 8:
just amaz eat perfect definit thing becaus like onli wait ca love delici come time
Topic 9:
great time love alway come recommend food look nice like wait staff tabl becaus best
Topic 10:
like veri restaur fresh becaus nice did alway onli time food good place definit sandwich


In [462]:
review_LDA_good, bus_review_LDA_good = LDA_merge(bus_review_small, good_r_small, lda_x_good, LDA_cols)

In [466]:
bus_review_LDA_good.shape

(92, 40)

In [467]:
review_LDA_good.shape

(684, 45)

In [469]:
lda_x_good.shape

(1000, 10)

In [460]:
lda_x_good

,LDA_1,LDA_2,LDA_3,LDA_4,LDA_5,LDA_6,LDA_7,LDA_8,LDA_9,LDA_10
0,0.050000,0.050000,0.050000,0.549999,0.050000,0.050000,0.050000,0.050000,0.050001,0.050000
1,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
2,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
3,0.274998,0.025000,0.025000,0.025000,0.025000,0.025000,0.274981,0.025000,0.275022,0.025000
4,0.050000,0.550000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
5,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
6,0.025001,0.275778,0.025000,0.279427,0.025000,0.025000,0.025000,0.025000,0.025000,0.269793
7,0.025000,0.276108,0.025000,0.025000,0.025000,0.025000,0.025000,0.025000,0.273938,0.274954
8,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
9,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.550000,0.050000,0.050000


In [ ]:
no_topics = 10
no_top_words = 10

LDA_cols = ['LDA_' + str(i) for i in range(1, no_topics + 1)]

In [ ]:
# GOOD

In [420]:
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer_good = CountVectorizer(max_df=0.90, min_df=20, stop_words = my_stop_words)

tf_good = tf_vectorizer_good.fit_transform(good_r['stemmed2'])
tf_feature_names_good = tf_vectorizer_good.get_feature_names()


In [429]:
lda_good = LatentDirichletAllocation(n_components = no_topics, max_iter = 10, learning_method='online',
                                learning_offset=50., random_state = 10)


In [430]:
lda_fit_good = lda_good.fit(tf_good)

lda_x = pd.DataFrame(lda_good.transform(tf_good), columns = LDA_cols)


In [431]:
#display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda_fit_good, tf_feature_names_good, no_top_words=15)

Topic 1:
order wait make onli night perfect know day sure tabl star cook got ca walk
Topic 2:
like definit tast enjoy experi eat tasti littl busi veri roll good way use rice
Topic 3:
great food good servic drink veri pretti price say seat sushi awesom atmospher worth pork
Topic 4:
tri come came order time becaus meat sinc beef meal soup egg offer befor ask
Topic 5:
restaur bar excel menu favorit food open expect order item took actual becaus free old
Topic 6:
friend realli time just veri nice food staff alway good dinner went servic think like
Topic 7:
best chees did thing flavor better good wine dessert spici start quit bread appet grill
Topic 8:
place love lunch visit special area spot locat coffe year patio larg lot fun small
Topic 9:
amaz place dish bit peopl right happi new hour like review fish usual home attent
Topic 10:
delici fresh recommend everyth sandwich sweet serv high quick someth ani need just potato check


### Testing

In [5]:
from sklearn.externals import joblib

In [12]:
lda_all = joblib.load('lda_all.pkl')
lda_fit = joblib.load('lda_fit.pkl')
tf = joblib.load('tf.pkl')

tf_vectorizer = joblib.load('tf_vectorizer.pkl')

In [8]:
x = lda_all.transform(tf)

In [9]:
x

array([[ 0.07333336,  0.54000021,  0.00666667, ...,  0.00666667,
         0.00666667,  0.00666667],
       [ 0.025     ,  0.27500004,  0.025     , ...,  0.02501397,
         0.025     ,  0.025     ],
       [ 0.01      ,  0.01      ,  0.01000385, ...,  0.01      ,
         0.01      ,  0.01      ],
       ..., 
       [ 0.02000271,  0.02000287,  0.02      , ...,  0.02000295,
         0.41998943,  0.21999729],
       [ 0.02500371,  0.27498753,  0.025     , ...,  0.025     ,
         0.025     ,  0.275     ],
       [ 0.1       ,  0.1       ,  0.1       , ...,  0.1       ,
         0.1       ,  0.1       ]])

In [11]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print( "Topic %d:" % (topic_idx + 1))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[ :-no_top_words - 1:-1]]))


In [13]:
tf_feature_names = tf_vectorizer.get_feature_names()

#display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda_fit, tf_feature_names, no_top_words = 15)

Topic 1:
food want bar experi enjoy star place bad disappoint anoth expect noth small home restaur
Topic 2:
place love got definit night onli portion excel favorit wine open offer dine come sushi
Topic 3:
realli delici menu tast come everyth cook bread egg coffe overal item larg ok return
Topic 4:
like just did place eat look restaur think know ani said good serv make spot
Topic 5:
tri lunch recommend bit flavor fresh way locat right like rice high great special soup
Topic 6:
order good best dish thing better chees dinner lot tri busi sure meat new tasti
Topic 7:
wait becaus drink ask came tabl peopl minut befor place ca did hot just custom
Topic 8:
time meal pretti say good sweet sinc everi veri big quick long tri went day
Topic 9:
great servic veri friend nice littl good staff seat hour visit atmospher happi work restaur
Topic 10:
food price alway amaz server qualiti perfect select sit kind feel let water line room


In [ ]:
# BAD

In [435]:
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model

no_topics = 10

tf_vectorizer_bad = TfidfVectorizer(max_df=0.90, min_df=20, stop_words = my_stop_words)

tf_bad = tf_vectorizer_bad.fit_transform(bad_r['stemmed2'])
tf_feature_names_bad = tf_vectorizer_bad.get_feature_names()


In [436]:
no_topics

10

In [437]:
lda_bad = LatentDirichletAllocation(n_components = no_topics, max_iter = 10, learning_method='online',
                                learning_offset=50., random_state = 10)


In [438]:
lda_fit_bad = lda_bad.fit(tf_bad)

In [439]:
display_topics(lda_fit_bad, tf_feature_names_bad, no_top_words=15)

Topic 1:
order sandwich lunch decent noth locat went work got special came wrong quick food felt
Topic 2:
return pretti check roll plate final serv offer ask way bread dessert dish order guess
Topic 3:
okay decid worst point atmospher lot bit shrimp stop sushi reserv place decor food huge
Topic 4:
custom night best love worth sure hot let probabl open clean differ owner place chees
Topic 5:
come tabl ok busi drink just did order want ask told peopl right server waitress
Topic 6:
disappoint think experi whi averag eat horribl coffe food egg fine veri dine line servic
Topic 7:
bad minut time better review place wait mayb tri seat look food impress els bland
Topic 8:
good servic realli food veri great nice like flavor delici tast slow happi staff lack
Topic 9:
star know tasti dri fresh expect reason amaz half beef walk size rice guy quit
Topic 10:
price definit recommend wo overal portion small terribl need food qualiti servic ani place high


### Aggregate LDA Weights

In [346]:
no_topics = 30
LDA_cols = ['LDA_' + str(i) for i in range(1, no_topics + 1)]

LDA_cols + ['review_id']

['LDA_1',
 'LDA_2',
 'LDA_3',
 'LDA_4',
 'LDA_5',
 'LDA_6',
 'LDA_7',
 'LDA_8',
 'LDA_9',
 'LDA_10',
 'LDA_11',
 'LDA_12',
 'LDA_13',
 'LDA_14',
 'LDA_15',
 'LDA_16',
 'LDA_17',
 'LDA_18',
 'LDA_19',
 'LDA_20',
 'LDA_21',
 'LDA_22',
 'LDA_23',
 'LDA_24',
 'LDA_25',
 'LDA_26',
 'LDA_27',
 'LDA_28',
 'LDA_29',
 'LDA_30',
 'review_id']

In [347]:
review_LDA_agg = review_LDA.groupby('review_id').mean().reset_index()[['review_id'] + ['sent_cnt'] + LDA_cols]

In [348]:
review_LDA_agg.head()

,review_id,sent_cnt,LDA_1,LDA_2,LDA_3,LDA_4,LDA_5,LDA_6,LDA_7,LDA_8,...,LDA_21,LDA_22,LDA_23,LDA_24,LDA_25,LDA_26,LDA_27,LDA_28,LDA_29,LDA_30
0,--8NyycOe-LDQVJoPyLy4g,6.0,0.038059,0.017210,0.013648,0.050880,0.060281,0.013648,0.004389,0.013648,...,0.004389,0.028321,0.045182,0.026611,0.041763,0.121108,0.056578,0.017210,0.004389,0.019541
1,--GIvp1-Y6--o1IlSTf7mQ,10.0,0.056522,0.022237,0.022237,0.033262,0.005570,0.022237,0.042237,0.005570,...,0.005570,0.069215,0.005570,0.050808,0.022237,0.005570,0.042933,0.038262,0.047548,0.013262
2,--J4sysENayuet-G4Q3D_g,3.0,0.006032,0.053651,0.006032,0.072698,0.072698,0.006032,0.053651,0.072698,...,0.006032,0.120317,0.006032,0.006032,0.006032,0.139365,0.006032,0.006032,0.006032,0.006032
3,--NJTPvefTMW0R2z7SJB1w,3.0,0.005309,0.042346,0.005309,0.071975,0.071975,0.042346,0.005309,0.060864,...,0.042346,0.005309,0.005309,0.005309,0.071975,0.005309,0.071975,0.005309,0.005309,0.153457
4,--Q_lxe0kqc6jk9eRSsH2g,4.0,0.088535,0.005202,0.027929,0.146111,0.146869,0.027929,0.005202,0.046869,...,0.069596,0.046869,0.050657,0.005202,0.138535,0.005202,0.005202,0.027929,0.046869,0.005202


In [440]:
bus_review_small.columns

Index(['user_id', 'business_id', 'name', 'categories', 'attributes', 'stars',
       'review_count', 'city', 'state', 'postal_code', 'latitude', 'longitude',
       'categories2', 'name_cnt', 'Sushi', 'Pizza', 'Restaurants', 'Cat_Flag',
       'cool', 'date', 'funny', 'review_id', 'user_stars', 'review_txt',
       'useful', 'yelping_since', 'first_yr_elite', 'elite_prior',
       'sent_token'],
      dtype='object')

In [441]:
# Aggregate LDA weights as a mean of the sentences
# Merge back to review df
def LDA_agg(df, cols):
    LDA_agg = df.groupby('review_id').mean().reset_index()[['review_id'] + ['sent_cnt'] + cols]
    bus_review_LDA = bus_review_small.set_index('review_id').join(LDA_agg.set_index('review_id'), how='inner')
    return bus_review_LDA
    
    

In [ ]:
bus_review_LDA = LDA_agg()

### Merge back to data

In [349]:
bus_review_LDA = bus_review_small.set_index('review_id').join(review_LDA_agg.set_index('review_id'), how='left')
bus_review_LDA.reset_index(inplace=True)

In [351]:
review_LDA_agg.columns

Index(['review_id', 'sent_cnt', 'LDA_1', 'LDA_2', 'LDA_3', 'LDA_4', 'LDA_5',
       'LDA_6', 'LDA_7', 'LDA_8', 'LDA_9', 'LDA_10', 'LDA_11', 'LDA_12',
       'LDA_13', 'LDA_14', 'LDA_15', 'LDA_16', 'LDA_17', 'LDA_18', 'LDA_19',
       'LDA_20', 'LDA_21', 'LDA_22', 'LDA_23', 'LDA_24', 'LDA_25', 'LDA_26',
       'LDA_27', 'LDA_28', 'LDA_29', 'LDA_30'],
      dtype='object')

In [352]:
bus_review_LDA.to_pickle('data/bus_review_LDA.pkl')

#### Create Flag for Useful / Funny

In [137]:
bus_review_LDA = np.load('data/bus_review_LDA.pkl')

In [356]:
# create Targets flags

bus_review_LDA['useful_flag'] = np.where(bus_review_LDA['useful'] >= 3, 1, 0)
bus_review_LDA['funny_flag'] = np.where(bus_review_LDA['funny'] >= 3, 1, 0)
bus_review_LDA['cool_flag'] = np.where(bus_review_LDA['cool'] >= 3, 1, 0)

In [357]:
bus_review_LDA.useful_flag.value_counts(normalize=True)

0    0.895304
1    0.104696
Name: useful_flag, dtype: float64

In [358]:
bus_review_LDA.columns

Index(['review_id', 'user_id', 'business_id', 'name', 'categories',
       'attributes', 'stars', 'review_count', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'categories2', 'name_cnt', 'Sushi', 'Pizza',
       'Restaurants', 'Cat_Flag', 'cool', 'date', 'funny', 'user_stars',
       'review_txt', 'useful', 'yelping_since', 'first_yr_elite',
       'elite_prior', 'sent_token', 'sent_cnt', 'LDA_1', 'LDA_2', 'LDA_3',
       'LDA_4', 'LDA_5', 'LDA_6', 'LDA_7', 'LDA_8', 'LDA_9', 'LDA_10',
       'LDA_11', 'LDA_12', 'LDA_13', 'LDA_14', 'LDA_15', 'LDA_16', 'LDA_17',
       'LDA_18', 'LDA_19', 'LDA_20', 'LDA_21', 'LDA_22', 'LDA_23', 'LDA_24',
       'LDA_25', 'LDA_26', 'LDA_27', 'LDA_28', 'LDA_29', 'LDA_30',
       'useful_flag', 'funny_flag', 'cool_flag'],
      dtype='object')

In [359]:
bus_review_LDA.head()

,review_id,user_id,business_id,name,categories,attributes,stars,review_count,city,state,...,LDA_24,LDA_25,LDA_26,LDA_27,LDA_28,LDA_29,LDA_30,useful_flag,funny_flag,cool_flag
0,DBFkxzPlSx0VBLdCGWPXpA,i8A3MWU33_mtMu3LUMlnWQ,s-YJpfR-LFRnH7qYVFuJuw,Touhenboku Ramen Restaurant,"[Japanese, Food, Restaurants, Ramen, Desserts]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,201,Toronto,ON,...,0.033223,0.020211,0.104262,0.060813,0.039310,0.006594,0.027317,0,0,0
1,SnwEJ7lAqGs6kZ2nbghudw,NMmu4oC-YRqdi0WY9pV78g,1zH3_Agw8CTgSmKgg4JNmg,Yasda Bento,"[Korean, Noodles, Japanese, Restaurants]","{'RestaurantsTableService': False, 'GoodForMea...",3.5,61,Phoenix,AZ,...,0.056402,0.101099,0.006402,0.006402,0.006402,0.006402,0.024259,0,0,0
2,6GcGV7Sq-axwgmPBdEy3fQ,T23h5oJqopYrLXp_A1GFYA,8XoDXN2DKmQGJHZhkQbJ9A,Emily's Cafe,"[Restaurants, Breakfast & Brunch, Cafes]","{'RestaurantsTableService': True, 'GoodForMeal...",3.5,79,Scottsdale,AZ,...,0.003164,0.003164,0.052244,0.031048,0.036497,0.036497,0.003164,0,0,0
3,0VX2IuiyGmC-LrLfYUnGeQ,bK-lus15ReXTyDqdEj4OQg,6nKR80xEGHYf2UxAe_Cu_g,The Thumb BBQ,"[Gas Stations, Burgers, Automotive, American (...","{'Alcohol': 'beer_and_wine', 'HasTV': True, 'N...",4.0,660,Scottsdale,AZ,...,0.009630,0.009630,0.009630,0.009630,0.009630,0.009630,0.009630,0,0,0
4,Cd3JHCfGtdYdMJ2_Ipqh6Q,uAf25oGZ8aURLI3u57GF_w,NPiLy71qqXzb45k9eQOgCQ,Maizie's Cafe & Bistro,"[Cafes, Restaurants, American (New)]","{'RestaurantsTableService': True, 'GoodForMeal...",4.0,382,Phoenix,AZ,...,0.040370,0.051749,0.040370,0.007037,0.007037,0.007037,0.007037,0,0,0


### Create models

In [180]:
#review_LDA.isnull().values.any()

In [181]:
#review_LDA.isnull().sum()

In [360]:
no_topics = 30
LDA_cols = ['LDA_' + str(i) for i in range(1, no_topics + 1)]

feature_cols = LDA_cols + ['user_stars'] + ['sent_cnt'] + ['elite_prior']
#  + ['sent_cnt']
feature_cols

['LDA_1',
 'LDA_2',
 'LDA_3',
 'LDA_4',
 'LDA_5',
 'LDA_6',
 'LDA_7',
 'LDA_8',
 'LDA_9',
 'LDA_10',
 'LDA_11',
 'LDA_12',
 'LDA_13',
 'LDA_14',
 'LDA_15',
 'LDA_16',
 'LDA_17',
 'LDA_18',
 'LDA_19',
 'LDA_20',
 'LDA_21',
 'LDA_22',
 'LDA_23',
 'LDA_24',
 'LDA_25',
 'LDA_26',
 'LDA_27',
 'LDA_28',
 'LDA_29',
 'LDA_30',
 'user_stars',
 'sent_cnt',
 'elite_prior']

In [361]:

X = bus_review_LDA[bus_review_LDA['useful'] >= 0][feature_cols]
y = bus_review_LDA[bus_review_LDA['useful'] >= 0]['funny_flag']

# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)


In [362]:
#X.isnull().sum()

In [363]:
logregM = LogisticRegression(C=1, penalty='l2', random_state=100, class_weight='balanced')

logregM.fit(X_train, y_train)

y_pred_train = logregM.predict_proba(X_train)
y_pred_test = logregM.predict_proba(X_test)
y_pred_test_class = logregM.predict(X_test)

fpr_rt_LR, tpr_rt_LR, _ = roc_curve(y_test, y_pred_test[:,1])

print ("Logistic Train ROC:", metrics.roc_auc_score(y_train, y_pred_train[:,1]))
print ("Logistic Test ROC:", metrics.roc_auc_score(y_test, y_pred_test[:,1]))

lr_auc = metrics.roc_auc_score(y_test, y_pred_test[:,1])

print(metrics.confusion_matrix(y_test, y_pred_test_class))
print("\nprecision", metrics.precision_score(y_test, y_pred_test_class))
print("recall", metrics.recall_score(y_test, y_pred_test_class))

print("avg precision:", average_precision_score(y_test, y_pred_test[:,1]))


Logistic Train ROC: 0.846342648241
Logistic Test ROC: 0.826052654607
[[11081  3365]
 [  148   401]]

precision 0.106479022836
recall 0.730418943534
avg precision: 0.178806759844


In [364]:
(list(zip(logregM.coef_[0], feature_cols)))

[(-1.5474224409579533, 'LDA_1'),
 (-1.1959038803020146, 'LDA_2'),
 (-1.1119465337757162, 'LDA_3'),
 (0.18368352674238272, 'LDA_4'),
 (-5.009462212972605, 'LDA_5'),
 (-3.1266745704854744, 'LDA_6'),
 (-0.77097662426623936, 'LDA_7'),
 (-4.4828116169391352, 'LDA_8'),
 (-1.3419099006011002, 'LDA_9'),
 (0.0165380516113218, 'LDA_10'),
 (-0.56988569265420552, 'LDA_11'),
 (-0.39741060928216537, 'LDA_12'),
 (1.6529711566655796, 'LDA_13'),
 (-0.75018563997422183, 'LDA_14'),
 (4.5912266458008295, 'LDA_15'),
 (2.0716489171493149, 'LDA_16'),
 (3.5767042775869049, 'LDA_17'),
 (-5.5308098215079182, 'LDA_18'),
 (1.3472259496970176, 'LDA_19'),
 (2.6309955858293486, 'LDA_20'),
 (1.8971956032056334, 'LDA_21'),
 (2.0819660553847452, 'LDA_22'),
 (0.70908779627210694, 'LDA_23'),
 (-1.4456669388517875, 'LDA_24'),
 (0.053613097936294013, 'LDA_25'),
 (-3.2213297676705754, 'LDA_26'),
 (0.012694423966149665, 'LDA_27'),
 (4.6640769601554517, 'LDA_28'),
 (2.8365401994824713, 'LDA_29'),
 (0.99367999329282597, 'LDA_3

In [366]:
Forest = RandomForestClassifier(n_estimators=500, max_depth = 4, min_samples_split=30, class_weight='balanced',
                                bootstrap=True, oob_score=True, random_state=100)

Forest.fit(X_train, y_train)
y_pred_train = Forest.predict_proba(X_train)
y_pred_test = Forest.predict_proba(X_test)
y_pred_test_class = Forest.predict(X_test)

fpr_rt_RF, tpr_rt_RF, _ = roc_curve(y_test, y_pred_test[:,1])

#print("RForest Accuracy: ", metrics.accuracy_score(y_test, y_pred_test))
print("RForest ROC train: ", metrics.roc_auc_score(y_train, y_pred_train[:,1]))
print("RForest ROC test: ", metrics.roc_auc_score(y_test, y_pred_test[:,1]))

rf_auc = metrics.roc_auc_score(y_test, y_pred_test[:,1])



print(metrics.confusion_matrix(y_test, y_pred_test_class))

print("\nprecision", metrics.precision_score(y_test, y_pred_test_class))
print("recall", metrics.recall_score(y_test, y_pred_test_class))

print("avg precision:", average_precision_score(y_test, y_pred_test[:,1]))

features = X_train.columns
pd.DataFrame({'feature': features, 'importance': Forest.feature_importances_}).\
sort_values('importance', ascending=False).reset_index(drop=True)


RForest ROC train:  0.856042746332
RForest ROC test:  0.821029362033
[[10940  3506]
 [  138   411]]

precision 0.104927240235
recall 0.748633879781
avg precision: 0.166792552871


,feature,importance
0,elite_prior,0.287231
1,sent_cnt,0.192545
2,LDA_28,0.072823
3,LDA_29,0.046715
4,LDA_22,0.045414
5,LDA_15,0.041279
6,LDA_17,0.036490
7,LDA_19,0.033983
8,LDA_23,0.030967
9,LDA_7,0.019743


In [88]:
review_LDA.funny_flag.value_counts()

0    313713
1    113025
Name: funny_flag, dtype: int64

In [ ]:
pd.concat([review_LDA.iloc[:, 1], review_LDA.iloc[:,24:]], axis=1)

In [ ]:
review_LDA.iloc[:, 1:]

### Plots

In [ ]:
review_LDA.columns

In [ ]:
review_LDA.columns[29:]

In [ ]:
for i in range(27, 48, 3):
    print(review_LDA.columns.tolist()[i:i+3])

In [ ]:
for i in range(27, 48, 3):
    sns.pairplot(review_LDA[review_LDA.columns.tolist()[i:i+3] + ['Pizza']], hue="Pizza", palette='coolwarm')

In [ ]:
for i in range(27, 48, 3):
    sns.pairplot(review_LDA[review_LDA.columns.tolist()[i:i+3] + ['city']], hue="city", palette='coolwarm')

#### Sentiment Analysis

In [ ]:
reviews_vect_df['polarity'] = reviews_vect_df['review_txt'].apply(lambda text: TextBlob(text).sentiment[0])
reviews_vect_df['subjectivity'] = reviews_vect_df['review_txt'].apply(lambda text: TextBlob(text).sentiment[1])

#TextBlob(reviews_vect_df.iloc[0]['review_txt']).sentiment

In [ ]:
TextBlob(reviews_vect_df.iloc[0]['review_txt']).sentiment

In [ ]:
pd.set_option('display.max_columns', 500)
reviews_vect_df.head()